### Age Detection of Indian Actors

In [ ]:
# Importing necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils
from tensorflow.keras.constraints import MaxNorm
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer, Dropout
import keras
import imageio # To read images
from PIL import Image # For image resizing 
from clr_callback import * # For cyclic learning rate

In [ ]:
# Reading the data
train = pd.read_csv('agedetectiontrain/train.csv')
test = pd.read_csv('agedetectiontest/test.csv')

In [ ]:
temp = []
for img_name in train.ID:
    img_path = os.path.join('agedetectiontrain/Train', img_name)
    img = imageio.imread(img_path)
    img = np.array(Image.fromarray(img).resize((32, 32))).astype('float32')    
    temp.append(img)

train_x = np.stack(temp)

In [ ]:
# Image resizing of test data into single numpy array
temp = []
for img_name in test.ID:
    img_path = os.path.join('agedetectiontest/Test', img_name)
    img = imageio.imread(img_path)
    img = np.array(Image.fromarray(img).resize((32, 32))).astype('float32')    
    temp.append(img)

test_x = np.stack(temp)

In [ ]:
# Normalizing the images
train_x = train_x / 255.
test_x = test_x / 255.

In [ ]:
# Encoding the categorical variable to numeric
lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.to_categorical(train_y)

### Building a Deep Neural Network for Age Group Detection

In [ ]:
# Specifying all the parameters we will be using in our network
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3

# Optimum values
optimizer = keras.optimizers.Adam()
activation = 'relu'
cb_triangular_lr = CyclicLR(base_lr=0.0001, max_lr=0.001, step_size=2000., mode='triangular2') 
weights = keras.initializers.he_normal()
bias = keras.initializers.Constant(value=0.01)
dropout = 0.30
regularizer = keras.regularizers.l2()
epochs = 21
batch_size = 128

In [ ]:
# Defining the network
model = Sequential([
    InputLayer(input_shape=input_num_units),
    Flatten(),
    Dense(units=hidden_num_units, kernel_initializer=weights, 
          bias_initializer=bias, activation=activation, 
          kernel_constraint=MaxNorm(3),  # Updated constraint
          kernel_regularizer=regularizer),
    Dropout(dropout),
    Dense(units=output_num_units, kernel_initializer=weights, activation='softmax'),
])

In [ ]:
# Printing model summary
model.summary()

In [ ]:
# Compiling and Training Network
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
cb_save = keras.callbacks.TensorBoard(log_dir='optimum_model', write_graph=False) 
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1, 
         validation_split=0.2, callbacks = [cb_triangular_lr, cb_save])

### Saving the model in H5 format

In [ ]:
model.save('optimum_model.h5')

In [ ]:
# Predicting and importing the result in a csv file
pred = model.predict_classes(test_x)
pred = lb.inverse_transform(pred)

test['Class'] = pred
test.to_csv('out.csv', index=False)

In [ ]:
# Visual Inspection of predictions
idx = 2481
img_name = test.ID[idx]

img = imageio.imread(os.path.join('agedetectiontest/Test', img_name))
plt.imshow(np.array(Image.fromarray(img).resize((128, 128))))
pred = model.predict_classes(test_x)
print('Original:', train.Class[idx], 'Predicted:', lb.inverse_transform(pred)[idx])